Objective: Expand NPS Predictive Model based on Flight Control/Treatment pairs to newer datasets and also to more apps than just Excel.

In [2]:
!pip install graphviz

You are using pip version 19.0.3, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
def odd_v_prob(p1,p2):
    '''representing change in probability and corresponding change in odds
    p1: initial probability
    p2: updated probability'''
    p_change = p2-p1
    o1 = p1/(1-p1)
    o2 = p2/(1-p2)
    o_change = o2-o1
    return p_change, o_change

print(odd_v_prob(0.1,0.6))
print(odd_v_prob(0.2,0.7))
print(odd_v_prob(0.3,0.8))

(0.5, 1.3888888888888886)
(0.49999999999999994, 2.083333333333333)
(0.5, 3.571428571428572)


In [2]:
import math
import pandas as pd
import numpy as np
import datetime
import sklearn.tree
from graphviz import Source
from IPython.display import SVG
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.feature_selection import RFE, SelectKBest, f_classif, mutual_info_classif, chi2,SelectPercentile,SelectFdr,SelectFpr,SelectFromModel
import os

from sklearn.linear_model import LogisticRegressionCV, LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.dummy import DummyClassifier

import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats

def transform_rating(rating):
    '''Input: Column of Data with NPS Field (on a scale of 1 to 5...)
    Output: Column with either Promoter/Detractor/Passive label, or the corresponding weights, based on datatype'''
    
    if rating == 5:
        return 100
    elif rating <= 3:
        return -100
    elif rating == 4:
        return 0
    else:
        return np.nan

In [7]:
#############  RUN  THIS  ##############
month = 'December'######################
month[:3]###############################

'Dec'

In [8]:
Word_df = pd.read_csv('Word_updated_data'+month[:3]+'.tsv', sep='\t')
Excel_df = pd.read_csv('Excel_updated_data'+month[:3]+'.tsv', sep='\t')
PP_df = pd.read_csv('PowerPoint_updated_data'+month[:3]+'.tsv', sep='\t')

C:\Users\v-Jaand\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\v-Jaand\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\v-Jaand\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (10,15,18,22,26,28,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
PP_df.head()

,OcvId,ProcessSessionId,Date,LongDate,Platform,Product,Rating,SurveyRatingScale,OriginalText,Verbatim,...,Skus,rn,FlightId,WacSessionId,WACSessionID,Application,ApplicationMode,ApplicationLCID,Host,rn1
0,flnps_v2_b59dd6b09c805661bc7face573961b82,477e30ec-a5ad-4052-af7a-5cbedcc7d96b,10/27/2019 2:04:04 PM,2019-10-27,Web,PowerPoint,5,5,"Facilita a edição, é prático e oferece bons de...","Facilitates editing, is practical and offers g...",...,NaN,1,canary-c,477e30ec-a5ad-4052-af7a-5cbedcc7d96b,477e30ec-a5ad-4052-af7a-5cbedcc7d96b,PowerPoint,Unified,pt-BR,OneDriveWOPI,1.0
1,flnps_v2_b3397471bc1158b9b359291a1da871ab,481cce98-912d-66ac-366b-81cb3731d61a,6/4/2019 1:34:26 PM,2019-06-04,Web,PowerPoint,3,5,NaN,NaN,...,NaN,1,canary-c,481cce98-912d-66ac-366b-81cb3731d61a,481cce98-912d-66ac-366b-81cb3731d61a,PowerPoint,Unified,en-US,SharePoint Online,1.0
2,flnps_v2_cb448186b6ad52809d8199a77b9772ae,4fd742a2-1196-486b-9518-68b8530a1fe4,5/4/2019 8:10:28 PM,2019-05-04,Web,PowerPoint,5,5,NaN,NaN,...,NaN,1,canary-t,4fd742a2-1196-486b-9518-68b8530a1fe4,4fd742a2-1196-486b-9518-68b8530a1fe4,PowerPoint,Unified,pt-BR,OneDriveWOPI,1.0
3,flnps_v2_6722df959f745987b2718d2ae4b17606,20aeb498-8576-4915-a6e1-fa4697445d3c,5/31/2019 7:55:06 PM,2019-05-31,Web,PowerPoint,5,5,NaN,NaN,...,NaN,1,canary-t,20aeb498-8576-4915-a6e1-fa4697445d3c,20aeb498-8576-4915-a6e1-fa4697445d3c,PowerPoint,Unified,en-US,SharePoint Online,1.0
4,flnps_v2_5eeca61dece15306ac756297f59dccbe,01339fec-c71d-4cd9-a486-22a0c14c5a49,7/1/2019 2:25:29 PM,2019-07-01,Web,PowerPoint,3,5,NaN,NaN,...,NaN,1,afd_ignorewaccluster,01339fec-c71d-4cd9-a486-22a0c14c5a49,01339fec-c71d-4cd9-a486-22a0c14c5a49,PowerPoint,Unified,en-US,OneDriveWOPI,1.0


In [10]:
def make_AB_dataset(df,prefixes):
    '''df: initial input dataframe of flight data
    prefixes: flights have prefixes - this arg is a list for which prefixes we want to filter on'''
    df = df[df['AudienceGroup']=='Production']
    df['Date'] = pd.to_datetime(df['Date'])
    flights = df.FlightId.astype(str).unique()
    controls = [f for f in flights if f.endswith('-c') or f.__contains__('control')]
    treatments = [f for f in flights if f.endswith('-t') or f.__contains__('treatment')]
    neither = [f for f in flights if (f not in controls) and (f not in treatments)]
    union = {'controls':[c.rstrip('control') for c in controls],'treatments':[t.rstrip('treatments') for t in treatments]}
    no_control = list(set(union['treatments'])-set(union['controls']))
    control_treatment_pairs = list(set(union['treatments'])-set(no_control))
    df['FlightPair']=df['FlightId'].astype(str).map(lambda x: x.rstrip('control'))
    df['FlightPair']=df['FlightPair'].map(lambda x: x.rstrip('treatment'))
    #df['FlightPair'].replace('docowner-canary','canary-docowner',inplace=True)
    
    
    ab_df = df[df.FlightId.notnull()]
    ab_df.drop_duplicates(keep='last', inplace=True)
    print(ab_df.shape, ' before filtering out non-pairs')
    ab_df = ab_df[ab_df['FlightPair'].isin(control_treatment_pairs)]
    print(ab_df.shape, ' after filtering out non-pairs')
    ab_df.loc[ab_df.FlightId.str.endswith('-c'),'Group'] = 'Control'
    ab_df.loc[ab_df.FlightId.str.endswith('control'),'Group'] = 'Control'
    ab_df.loc[ab_df.FlightId.str.endswith('-t'),'Group'] = 'Treatment'
    ab_df.loc[ab_df.FlightId.str.endswith('treatment'),'Group'] = 'Treatment'
    ab_df.loc[ab_df.FlightId.str.endswith('-c'),'Flight'] = 0
    ab_df.loc[ab_df.FlightId.str.endswith('control'),'Flight'] = 0
    ab_df.loc[ab_df.FlightId.str.endswith('-t'),'Flight'] = 1
    ab_df.loc[ab_df.FlightId.str.endswith('treatment'),'Flight'] = 1
    ab_df = ab_df[ab_df.Flight.notnull()]
    ab_df['NPS'] = ab_df['Rating'].apply(transform_rating)
    ab_df = ab_df.sort_values(by='Date')
    value_key = ab_df.sort_values(by='Date').groupby(['OcvId'])['NPS'].last().to_dict()
    exp_df = ab_df.groupby(['OcvId','FlightPair'])['Flight'].last().unstack()
    print('Feature Matrix should have ',ab_df.OcvId.nunique(), ' rows and ',ab_df.FlightPair.nunique(),' columns')
    print('Final Shape:',exp_df.shape)
    if prefixes: #i.e. if the input list is empty:
        for p in prefixes:
            exp_df = exp_df.iloc[:,exp_df.columns.str.startswith(p)]
    exp_df['NPS'] = exp_df.index.map(value_key)
    return exp_df.fillna(0)

#excel_df = make_AB_dataset(Excel_df,['xls'])

In [85]:
####### JUST READ THE CSV IF YOUVE RAN ALREADY #######
flight_durations = pd.read_csv('FlightsAsOf'+month+'.csv',index_col='FlightPair')

In [ ]:
def get_flight_durations():
    ''' no inputs/arguments, just make sure you have all of the Tabular Flight data files you are using.'''
    df = pd.concat([Excel_df[Excel_df['AudienceGroup']=='Production'][Excel_df.FlightId.notnull()],
                    Word_df[Word_df['AudienceGroup']=='Production'][Word_df.FlightId.notnull()],
                    PP_df[PP_df['AudienceGroup']=='Production'][PP_df.FlightId.notnull()]])
    #df = df[df['AudienceGroup']=='Production']
    df['Date'] = pd.to_datetime(df['Date'])
    flights = df.FlightId.astype(str).unique()
    controls = [f for f in flights if f.endswith('-c') or f.__contains__('control')]
    treatments = [f for f in flights if f.endswith('-t') or f.__contains__('treatment')]
    neither = [f for f in flights if (f not in controls) and (f not in treatments)]
    union = {'controls':[c.rstrip('control') for c in controls],'treatments':[t.rstrip('treatments') for t in treatments]}
    no_control = list(set(union['treatments'])-set(union['controls']))
    control_treatment_pairs = list(set(union['treatments'])-set(no_control))
    df['FlightPair']=df['FlightId'].astype(str).map(lambda x: x.rstrip('control'))
    df['FlightPair']=df['FlightPair'].map(lambda x: x.rstrip('treatment'))
    
    print('Flight Pairs Assigned')
    
    ab_df = df[df.FlightPair.notnull()]
    #ab_df.drop_duplicates(keep='last', inplace=True)
    ab_df = ab_df[ab_df['FlightPair'].isin(control_treatment_pairs)]
    flight_starts = ab_df.sort_values(by='Date').groupby(['FlightPair'])['Date'].first()
    flight_ends = ab_df.sort_values(by='Date').groupby(['FlightPair'])['Date'].last()
    flight_counts = ab_df.groupby(['FlightPair'])['Date'].count()
    flight_durations = pd.concat([flight_starts,flight_ends,flight_counts],axis=1)
    flight_durations.columns = ['FlightStart','FlightEnd','FlightCounts']
    return flight_durations
    
flight_durations = get_flight_durations()

In [86]:
flight_durations.head(3)

,FlightStart,FlightEnd,FlightCounts
FlightPair,,,
allapps-updatedcontextualtabvisualscomposed-,2019-12-11 18:25:15,2019-12-30 23:44:09,2467
box4complextables-,2019-05-16 00:19:19,2019-08-06 20:59:14,3094
canary-,2019-05-01 00:04:11,2019-12-30 23:59:06,115110


In [9]:
flight_durations.to_csv('FlightsAsOf'+month+'.csv')

In [13]:
excel_df = make_AB_dataset(Excel_df,[])
word_df = make_AB_dataset(Word_df,[])
pp_df = make_AB_dataset(PP_df,[])

C:\Users\v-Jaand\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(1924893, 40)  before filtering out non-pairs
(1554621, 40)  after filtering out non-pairs
Feature Matrix should have  40938  rows and  194  columns
Final Shape: (40938, 194)
(2834381, 40)  before filtering out non-pairs
(2271826, 40)  after filtering out non-pairs
Feature Matrix should have  41530  rows and  336  columns
Final Shape: (41530, 336)


C:\Users\v-Jaand\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\v-Jaand\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\v-Jaand\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

(762974, 40)  before filtering out non-pairs
(476359, 40)  after filtering out non-pairs
Feature Matrix should have  29772  rows and  72  columns
Final Shape: (29772, 72)


In [14]:
excel_flights = list(excel_df.columns)
word_flights = list(word_df.columns)
common_flights = list(set(excel_flights).intersection(word_flights))
pp_flights = list(pp_df.columns)
common_flights = list(set(common_flights).intersection(pp_flights))

In [15]:
common_flights.remove('NPS')

In [16]:
common_flights

['wac-wordimagegroupcontainerfittopage-',
 'wac-wordpreserveverbonredirect-',
 'firstrelease-',
 'wac-wordcloneatmentionpropertiesinhyperlink-',
 'wac-licensingisenabled-',
 'wac-wordsdxpreloadpackagelist-',
 'wac-wordwordoauthtestappaadtokenretrieval-',
 'docowner-canary-',
 'wac-wordrendervisibleoutlinegroupsperf-',
 'wac-wordshowpasteoptionsincontextmenu-',
 'wac-worddeprioritizerichtextcontentcontrolacetateforcoauth-',
 'wacdash-',
 'wac-oauthmsatokenretrieval-',
 'canary2-',
 'wac-worddefinenewbullettablecellclassname-',
 'wac-wordwordoauthtestappmsatokenretrieval-',
 'wac-worduseaadforfileupload-',
 'wac-mergemissingeopfix-',
 'xls-commitandcanceleditbuttonsenabled-',
 'xls-sheettabbarcontroltouchscroll-',
 'useridtestaa-',
 'wac-box4directprinthidesensitiveparameters-',
 'xls-customsortforallrangetypes-',
 'xls-removescrolltofirstandlastsheettabnavbuttons-',
 'wac-wordaddfontserviceresponselength-',
 'wac-oauthtestappmsatokenretrieval-',
 'xls-overridebrowsershortcuts-',
 'docow

In [17]:
dc = ['docowner-canary-','canary-docowner-',
     'canary2','canary-']

In [18]:
X = {'excel':excel_df.iloc[:,excel_df.columns.str.startswith('xls')],
     'word':word_df.iloc[:,word_df.columns.str.startswith('wac')],
     'pp':pp_df.iloc[:,pp_df.columns.str.startswith('pp')],
     'all':pd.concat([excel_df,word_df,pp_df]).loc[:,common_flights]}
y = {'excel':excel_df['NPS'].replace([100,0],1).replace(-100,0)
     ,'word':word_df['NPS'].replace([100,0],1).replace(-100,0),
     'pp':pp_df['NPS'].replace([100,0],1).replace(-100,0),
     'all':pd.concat([excel_df,word_df,pp_df])['NPS'].replace([100,0],1).replace(-100,0)}

C:\Users\v-Jaand\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.
C:\Users\v-Jaand\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [19]:
print(X['excel'].shape, y['excel'].shape)
print(X['word'].shape, y['word'].shape)
print(X['pp'].shape, y['pp'].shape)
print(X['all'].shape, y['all'].shape)

(40938, 144) (40938,)
(41530, 305) (41530,)
(29772, 21) (29772,)
(112240, 41) (112240,)


In [15]:
model_perfs = pd.DataFrame(columns=['Logistic','RandomForest','GradientBoosting'])
for key in ['excel','word','pp','all']:
########## WHEN PREDICTORS ARE BINARY (1 for Treatment, 0 otherwise) ###############
    logit = LogisticRegression().fit(X[key],y[key])
    rf = RandomForestClassifier(random_state=0).fit(X[key],y[key])
    gb = xgb.XGBClassifier(random_state=0, n_jobs=4).fit(X[key],y[key])
    
    log_acc = round(logit.score(X[key],y[key]),3)
    rf_acc = round(rf.score(X[key],y[key]),3)
    gb_acc = round(gb.score(X[key],y[key]),3)

    model_perfs.loc[key+'Accuracy']=[log_acc,rf_acc,gb_acc]

C:\Users\v-Jaand\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\v-Jaand\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\v-Jaand\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\v-Jaand\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\v-Jaand\AppDa

In [16]:
model_perfs

,Logistic,RandomForest,GradientBoosting
excelAccuracy,0.688,0.865,0.689
wordAccuracy,0.690,0.969,0.690
ppAccuracy,0.740,0.741,0.740
allAccuracy,0.702,0.753,0.702


## Excel

In [157]:
X_train, X_test, y_train, y_test = train_test_split(X['excel'], y['excel'], test_size=0.2)

In [18]:
### ONLY RUN IF THERE IS NO OUTPUT ###
rf = RandomForestClassifier(random_state=0)
param_grid = { 
    'n_estimators': [50, 100, 250, 500],
    #'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [2,5,10,25,None]}
    #'oob_score': [True,False]}

CV_rfc = GridSearchCV(estimator=rf, n_jobs=-1, param_grid=param_grid, verbose=10, scoring='neg_log_loss',cv= 5)
CV_rfc.fit(X_train, y_train)
print(CV_rfc.best_params_, CV_rfc.best_score_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   16.4s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   21.3s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   39.3s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   47.5s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 317.4min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 322.0min finished


{'max_depth': 10, 'n_estimators': 500} -0.6044165697512734


In [181]:
print(RandomForestClassifier(random_state=0).fit(X_train,
                                           y_train).score(X_test,y_test))


rfc = RandomForestClassifier(random_state = 0, n_estimators = 500,
                      max_depth=10,oob_score=True)
rfc.fit(X_train,y_train)
print(rfc.score(X_test,y_test))

learners = rfc.feature_importances_.argsort()[::-1]

features = pd.DataFrame(X_train.columns[learners], rfc.feature_importances_[learners])
features = features[features.index>0.015]
tree_best = features.FlightPair.values.tolist()

C:\Users\v-Jaand\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.6314118221787982
0.6822178798241328


In [165]:
features[features.FlightPair.str.startswith('xls-slr')]

,FlightPair
0.009461,xls-slrcachegalleries-
0.008811,xls-slrcachecolorpickers-
0.007379,xls-slrcachecomboboxmenus-


In [169]:
flight_durations[flight_durations.index.str.startswith('xls-slr')] #normally SLR Cache Galleries is a strong feature, but not in this iteration.

,FlightStart,FlightEnd,FlightCounts
FlightPair,,,
xls-slrcachecolorpickers-,2019-05-10 00:45:10,2019-12-30 23:59:06,33133
xls-slrcachecomboboxmenus-,2019-05-01 16:00:05,2019-12-30 23:59:06,37316
xls-slrcachegalleries-,2019-05-31 18:09:09,2019-12-30 23:59:06,31769


In [ ]:
print(LogisticRegression().fit(X_train,y_train).score(X_test,y_test), ' -->...')

for i in [1,5,15,25,50]:
    print('Score with Top ',i,' Features:')
    logit = LogisticRegression(solver='liblinear')

    rfe = RFE(logit,i)
    rfe = rfe.fit(X_train,y_train.values.ravel())

    rfe.support_
    rfe.ranking_

    #identified columns Recursive Feature Elimination
    idc_rfe = pd.DataFrame({"rfe_support" :rfe.support_,
                           "columns" : [i for i in X_train.columns],
                           "ranking" : rfe.ranking_,
                          })
    cols = idc_rfe[idc_rfe["rfe_support"] == True]["columns"].tolist()


    #separating train and test data
    train_rfe_X = X_train[cols]
    train_rfe_y = y_train.copy()

    test_rfe_X = X_test[cols]
    test_rfe_y = y_test.copy()


    logit_rfe = LogisticRegression(class_weight=None, dual=False, fit_intercept=True,
              intercept_scaling=1, max_iter=100, #n_jobs=4,
              penalty='l2', random_state=None, solver='liblinear', #tol=0.0001,
              verbose=0, warm_start=False)

    logit_rfe.fit(train_rfe_X, train_rfe_y)
    print(logit_rfe.score(test_rfe_X,test_rfe_y))

C:\Users\v-Jaand\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.686248168050806  -->...
Score with Top  1  Features:


In [ ]:
print(logit.fit(X_train,y_train).score(X_test,y_test), ' -->...')
logit_rfe.score(test_rfe_X,test_rfe_y)

In [170]:
logit = LogisticRegression(solver='liblinear')

rfe = RFE(logit,5)
rfe = rfe.fit(X_train,y_train.values.ravel())

rfe.ranking_

#identified columns Recursive Feature Elimination
idc_rfe = pd.DataFrame({"rfe_support" :rfe.support_,
                       "columns" : [i for i in X_train.columns],
                       "ranking" : rfe.ranking_,
                      })

rfe_best = idc_rfe[idc_rfe["rfe_support"] == True]["columns"].tolist()

In [171]:
fs = SelectKBest(f_classif,k=10)
fit = fs.fit(X['excel'],y['excel'])
k_bestF = list(X['excel'].columns[fit.get_support()])

In [172]:
fs = SelectKBest(chi2,k=10)
fit = fs.fit(X['excel'],y['excel'])
k_bestCHI = list(X['excel'].columns[fit.get_support()])

In [182]:
print(tree_best)
print(rfe_best)
print(k_bestF)
print(k_bestCHI)

['xls-wac-', 'xls-usezonereallocforclonesaveenabled-', 'xls-ideas-', 'xls-licensingisenabled-', 'xls-mergedecoupled-', 'xls-applyautomaticpolicylabel-', 'xls-linkedentityblankcellconvertteachingcalloutvideowithtext-', 'xls-ocpsisenabled-', 'xls-ribbonresourcespreloading-', 'xls-cardviewshowricherrors-', 'xls-nativeopenworkbookmeasurements-', 'xls-openworkbookdecoupled-', 'xls-tabletextcontrastacccheck-', 'xls-namedsheetviewspassivetest-', 'xls-xlowatermarkroundtrip-', 'xls-editablelegacycomments-', 'xls-earlysessionstartforview-']
['xls-automatetab-', 'xls-gridcanvasrendering-', 'xls-licensingisenabled-', 'xls-linkedentityblankcellconvertteachingcalloutvideowithtext-', 'xls-reconnectsessiononuserinteration-']
['xls-applyautomaticpolicylabel-', 'xls-cardviewshowricherrors-', 'xls-earlysessionstartforview-', 'xls-licensingisenabled-', 'xls-mergedecoupled-', 'xls-namedsheetviewspassivetest-', 'xls-ocpsisenabled-', 'xls-usezonereallocforclonesaveenabled-', 'xls-wac-', 'xls-xlowatermarkroun

In [191]:
cols = tree_best + rfe_best + k_bestF + k_bestCHI 
cols = list(set(cols))

In [192]:
len(cols)
cols.extend(['xls-slrcachegalleries-','xls-slrcachecomboboxmenus-'])

In [193]:
#redundant columns check
matrix = X['excel'].loc[:,cols].corr()==1
linear_dependents = []
for col in matrix.columns:
    #print(col)
    linear_dependents.append(list(matrix[col].index[matrix[col]==True]))
[x for x in linear_dependents if len(x)>1]

[['xls-licensingisenabled-', 'xls-wac-'],
 ['xls-licensingisenabled-', 'xls-wac-']]

In [194]:
#opposite columns check
matrix = X['excel'].loc[:,cols].corr()==-1
linear_combinations = []
for col in matrix.columns:
    #print(col)
    linear_combinations.append(list(matrix[col].index[matrix[col]==True]))
[x for x in linear_combinations if len(x)>1]

[]

In [195]:
X_excel = sm.add_constant(X['excel'].loc[:,cols]).T.drop_duplicates().T

In [196]:
print('first value of each line should always be second vale minus 1')
rank_test = X_excel
for i in range(rank_test.shape[1]):
    df_to_rank = rank_test.iloc[:,:i+1]
    print(i,np.linalg.matrix_rank(df_to_rank))

first value of each line should always be second vale minus 1
0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10
10 11
11 12
12 13
13 14
14 15
15 16
16 17
17 18
18 19
19 20
20 21
21 22
22 23
23 24


In [197]:
X_excel

,const,xls-usenewviewportcalculationsforprefetch-,xls-cardviewshowricherrors-,xls-nativeopenworkbookmeasurements-,xls-editablelegacycomments-,xls-licensingisenabled-,xls-mergedecoupled-,xls-earlysessionstartforview-,xls-ribbonresourcespreloading-,xls-gridcanvasrendering-,...,xls-automatetab-,xls-usezonereallocforclonesaveenabled-,xls-newcopypasteexperienceacrosssessions-,xls-reconnectsessiononuserinteration-,xls-applyautomaticpolicylabel-,xls-ocpsisenabled-,xls-namedsheetviewspassivetest-,xls-linkedentityblankcellconvertteachingcalloutvideowithtext-,xls-slrcachegalleries-,xls-slrcachecomboboxmenus-
OcvId,,,,,,,,,,,,,,,,,,,,,
flnps_v2_0000d4e32b3b50fe8ed3f50c0e36feb3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
flnps_v2_00017f19ec855c33b3e3ec4c4294a106,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
flnps_v2_0002e4096d8557cd84516b46d5a84fdc,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
flnps_v2_0007c41758d95e57a1af813fca5a0b33,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
flnps_v2_00082a1ce662513b8608b444f122ad2f,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
flnps_v2_fff4f907dcf55f1cb51fde8abddc6437,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
flnps_v2_fff841fb2822523d9b0784b9718d903f,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
flnps_v2_fffa3bc4fa305707bb60f8c6de1b94d1,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0


In [198]:
logit = sm.Logit(y['excel'],X_excel)

flogit = logit.fit()
print(flogit.summary())

coefficients = flogit.summary2().tables[1]
coefficients = coefficients[coefficients['P>|z|']<0.1]
coefficients['Odds Ratio']=np.exp(coefficients['Coef.'])
coefficients['O.R.LB']=np.exp(coefficients['[0.025'])
coefficients['O.R.UB']=np.exp(coefficients['0.975]'])
coefficients['FinalProbability'] = np.exp(coefficients['Coef.'].round(1))*y['excel'].value_counts(normalize=True).loc[1] #- 0.5
coefficients['FinalProbability'] = coefficients['FinalProbability'].mask(coefficients['FinalProbability']>=1,0.99)
coefficients['Lift'] = coefficients['FinalProbability'] - y['excel'].value_counts(normalize=True).loc[1]
excel_coefficients = coefficients.join(flight_durations)
excel_coefficients['Type'] = 'Excel'

         Current function value: 0.604515
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:                    NPS   No. Observations:                40938
Model:                          Logit   Df Residuals:                    40914
Method:                           MLE   Df Model:                           23
Date:                Thu, 16 Jan 2020   Pseudo R-squ.:                 0.02628
Time:                        09:35:46   Log-Likelihood:                -24748.
converged:                      False   LL-Null:                       -25416.
Covariance Type:            nonrobust   LLR p-value:                2.915e-268
                                                                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------
const                                     

C:\Users\v-Jaand\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [88]:
selects = {}
for f in excel_coefficients.index:
    selected_by = []
    for s,n in zip([tree_best,rfe_best,k_bestF,k_bestCHI],['RandomForest','Recursive','F','CHI2']):
        if f in s:
            selected_by.append(n)
        selects[f] = selected_by

excel_coefficients['SelectedBy'] = excel_coefficients.index.map(selects)

## Word

In [92]:
X_train, X_test, y_train, y_test = train_test_split(X['word'], y['word'], test_size=0.2)

In [34]:
rf = RandomForestClassifier(random_state=0)
param_grid = { 
    'n_estimators': [50, 100, 250, 500],
    #'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [2,5,10,25,None]}
    #'oob_score': [True,False]}

CV_rfc = GridSearchCV(estimator=rf, n_jobs=-1, param_grid=param_grid, verbose=10, scoring='neg_log_loss',cv= 5)
CV_rfc.fit(X_train, y_train)
print(CV_rfc.best_params_, CV_rfc.best_score_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   16.7s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   20.9s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   35.2s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   43.6s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 13.2min finished


{'max_depth': 10, 'n_estimators': 500} -0.6083596964701947


In [35]:
CV_rfc

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False, random_state=0,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'max

In [93]:
print(RandomForestClassifier(random_state=0).fit(X_train,
                                           y_train).score(X_test,y_test))
rfc = RandomForestClassifier(random_state = 0, n_estimators = 500,
                      max_depth=10,oob_score=True)

rfc.fit(X_train,y_train)
rfc.score(X_test,y_test)

C:\Users\v-Jaand\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.6325547796773416


0.6929930170960751

In [107]:
learners = rfc.feature_importances_.argsort()[::-1]
features = pd.DataFrame(X_train.columns[learners], rfc.feature_importances_[learners])
features = features[features.index>0.0125]
tree_best = features.FlightPair.values.tolist()
features

,FlightPair
0.022136,wac-wordaugloopoperations-
0.022006,wac-box4augloopnodeeventsfiringinterval-500ms-
0.021818,wac-wordstatefulaugloop-
0.019177,wac-box4augloopannotations-
0.018676,wac-wordideas-
0.018152,wac-box4augloopannotationsattaching-
0.017550,wac-wordaugloopserversideseeding-
0.017284,wac-wordprefetchideasresources-
0.016528,wac-wordcomboprefetchideasassets-
0.015295,wac-wordprefetchofficejsbetaresources-


In [38]:
print(LogisticRegression().fit(X_train,y_train).score(X_test,y_test), ' -->...')

for i in [1,5,10,15,25,50]:
    print('Score with Top ',i,' Features:')
    logit = LogisticRegression(solver='liblinear')

    rfe = RFE(logit,i)
    rfe = rfe.fit(X_train,y_train.values.ravel())

    rfe.support_
    rfe.ranking_

    #identified columns Recursive Feature Elimination
    idc_rfe = pd.DataFrame({"rfe_support" :rfe.support_,
                           "columns" : [i for i in X_train.columns],
                           "ranking" : rfe.ranking_,
                          })
    cols = idc_rfe[idc_rfe["rfe_support"] == True]["columns"].tolist()


    #separating train and test data
    train_rfe_X = X_train[cols]
    train_rfe_y = y_train.copy()

    test_rfe_X = X_test[cols]
    test_rfe_y = y_test.copy()


    logit_rfe = LogisticRegression(class_weight=None, dual=False, fit_intercept=True,
              intercept_scaling=1, max_iter=100, #n_jobs=4,
              penalty='l2', random_state=None, solver='liblinear', #tol=0.0001,
              verbose=0, warm_start=False)

    logit_rfe.fit(train_rfe_X, train_rfe_y)
    print(logit_rfe.score(test_rfe_X,test_rfe_y))

C:\Users\v-Jaand\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6929930170960751  -->...
Score with Top  1  Features:
0.695160125210691
Score with Top  2  Features:
0.695160125210691
Score with Top  3  Features:
0.695160125210691
Score with Top  4  Features:
0.695160125210691
Score with Top  5  Features:
0.695160125210691
Score with Top  10  Features:
0.695160125210691
Score with Top  15  Features:
0.695160125210691
Score with Top  20  Features:
0.6949193354201781
Score with Top  25  Features:
0.6950397303154346
Score with Top  50  Features:
0.6950397303154346
Score with Top  75  Features:
0.6945581507344089
Score with Top  100  Features:
0.6944377558391525


In [95]:
logit = LogisticRegression(solver='liblinear')

rfe = RFE(logit,5)
rfe = rfe.fit(X_train,y_train.values.ravel())

rfe.ranking_

#identified columns Recursive Feature Elimination
idc_rfe = pd.DataFrame({"rfe_support" :rfe.support_,
                       "columns" : [i for i in X_train.columns],
                       "ranking" : rfe.ranking_,
                      })

rfe_best = idc_rfe[idc_rfe["rfe_support"] == True]["columns"].tolist()

In [96]:
fs = SelectKBest(f_classif,k=10)
fit = fs.fit(X['word'],y['word'])
k_bestF = list(X['word'].columns[fit.get_support()])

In [97]:
fs = SelectKBest(chi2,k=10)
fit = fs.fit(X['word'],y['word'])
k_bestCHI = list(X['word'].columns[fit.get_support()])

In [113]:
cols = tree_best + rfe_best + k_bestF + k_bestCHI 
cols = list(set(cols))
len(cols)

22

In [114]:
#redundant columns check
matrix = (X['word'].loc[:,cols]).corr()==1
linear_dependents = []
for col in matrix.columns:
    #print(col)
    linear_dependents.append(list(matrix[col].index[matrix[col]==True]))
[x for x in linear_dependents if len(x)>1]

[['wac-wordprefetchofficejsbetaresources-',
  'wac-wordprefetchideasresources-',
  'wac-wordcomboprefetchideasassets-'],
 ['wac-wordprefetchofficejsbetaresources-',
  'wac-wordprefetchideasresources-',
  'wac-wordcomboprefetchideasassets-'],
 ['wac-wordaugloopserversideseeding-',
  'wac-box4augloopannotationsattaching-',
  'wac-wordstatefulaugloop-',
  'wac-box4augloopannotations-',
  'wac-wordaugloopoperations-'],
 ['wac-wordprefetchofficejsbetaresources-',
  'wac-wordprefetchideasresources-',
  'wac-wordcomboprefetchideasassets-'],
 ['wac-wordaugloopserversideseeding-',
  'wac-box4augloopannotationsattaching-',
  'wac-wordstatefulaugloop-',
  'wac-box4augloopannotations-',
  'wac-wordaugloopoperations-'],
 ['wac-wordaugloopserversideseeding-',
  'wac-box4augloopannotationsattaching-',
  'wac-wordstatefulaugloop-',
  'wac-box4augloopannotations-',
  'wac-wordaugloopoperations-'],
 ['wac-wordaugloopserversideseeding-',
  'wac-box4augloopannotationsattaching-',
  'wac-wordstatefulaugloo

In [115]:
#opposite columns check
matrix = (X['word'].loc[:,cols]).corr()==-1
linear_combinations = []
for col in matrix.columns:
    #print(col)
    linear_combinations.append(list(matrix[col].index[matrix[col]==True]))
[x for x in linear_combinations if len(x)>1]

[]

In [116]:
X_word = sm.add_constant(X['word'].loc[:,cols]).T.drop_duplicates().T

In [117]:
print('first value of each line should always be second vale minus 1')
rank_test = X_word
for i in range(rank_test.shape[1]):
    df_to_rank = rank_test.iloc[:,:i+1]
    print(i,np.linalg.matrix_rank(df_to_rank))

first value of each line should always be second vale minus 1
0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10
10 11
11 12
12 13
13 14
14 15
15 16
16 17


In [118]:
logit = sm.Logit(y['word'],X_word)
flogit = logit.fit()
print(flogit.summary())
coefficients = flogit.summary2().tables[1]
coefficients = coefficients[coefficients['P>|z|']<0.1]
coefficients['Odds Ratio']=np.exp(coefficients['Coef.'])
coefficients['O.R.LB']=np.exp(coefficients['[0.025'])
coefficients['O.R.UB']=np.exp(coefficients['0.975]'])
coefficients['FinalProbability'] = np.exp(coefficients['Coef.'].round(1))*y['word'].value_counts(normalize=True).loc[1] #- 0.5
coefficients['FinalProbability'] = coefficients['FinalProbability'].mask(coefficients['FinalProbability']>=1,0.99)
coefficients['Lift'] = coefficients['FinalProbability'] - y['word'].value_counts(normalize=True).loc[1]
word_coefficients = coefficients.join(flight_durations)
word_coefficients['Type'] = 'Word'

Optimization terminated successfully.
         Current function value: 0.607392
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                    NPS   No. Observations:                41530
Model:                          Logit   Df Residuals:                    41513
Method:                           MLE   Df Model:                           16
Date:                Wed, 15 Jan 2020   Pseudo R-squ.:                 0.02063
Time:                        17:22:32   Log-Likelihood:                -25225.
converged:                       True   LL-Null:                       -25756.
Covariance Type:            nonrobust   LLR p-value:                3.809e-216
                                                             coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------
const               

In [119]:
selects = {}
for f in word_coefficients.index:
    selected_by = []
    for s,n in zip([tree_best,rfe_best,k_bestF,k_bestCHI],['RandomForest','Recursive','F','CHI2']):
        if f in s:
            selected_by.append(n)
        selects[f] = selected_by

word_coefficients['SelectedBy'] = word_coefficients.index.map(selects)

## PowerPoint

In [120]:
print('PowerPoint AllUp MODEL')

PowerPoint AllUp MODEL


In [121]:
#redundant columns check
matrix = (X['pp']).corr()==1
linear_dependents = []
for col in matrix.columns:
    #print(col)
    linear_dependents.append(list(matrix[col].index[matrix[col]==True]))
[x for x in linear_dependents if len(x)>1]

[]

In [122]:
#opposite columns check
matrix = (X['pp']).corr()==-1
linear_combinations = []
for col in matrix.columns:
    #print(col)
    linear_combinations.append(list(matrix[col].index[matrix[col]==True]))
[x for x in linear_combinations if len(x)>1]

[]

In [123]:
rank_test = (sm.add_constant(X['pp'].T.drop_duplicates().T))
for i in range(rank_test.shape[1]):
    df_to_rank = rank_test.iloc[:,:i+1]
    print(i,np.linalg.matrix_rank(df_to_rank))

0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10
10 11
11 12
12 13
13 14
14 15
15 16
16 17
17 18
18 19
19 20
20 21
21 22


In [124]:
logit = sm.Logit(y['pp'],sm.add_constant(X['pp']))
flogit = logit.fit()
print(flogit.summary())
coefficients = flogit.summary2().tables[1]
coefficients = coefficients[coefficients['P>|z|']<0.1]
coefficients['Odds Ratio']=np.exp(coefficients['Coef.'])
coefficients['O.R.LB']=np.exp(coefficients['[0.025'])
coefficients['O.R.UB']=np.exp(coefficients['0.975]'])
coefficients['FinalProbability'] = np.exp(coefficients['Coef.'].round(1))*y['pp'].value_counts(normalize=True).loc[1] #- 0.5
coefficients['FinalProbability'] = coefficients['FinalProbability'].mask(coefficients['FinalProbability']>=1,0.99)
coefficients['Lift'] = coefficients['FinalProbability'] - y['pp'].value_counts(normalize=True).loc[1]
pp_coefficients = coefficients.join(flight_durations)
pp_coefficients['Type'] = 'PowerPoint'

Optimization terminated successfully.
         Current function value: 0.568304
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                    NPS   No. Observations:                29772
Model:                          Logit   Df Residuals:                    29750
Method:                           MLE   Df Model:                           21
Date:                Wed, 15 Jan 2020   Pseudo R-squ.:                0.008493
Time:                        17:23:05   Log-Likelihood:                -16920.
converged:                       True   LL-Null:                       -17064.
Covariance Type:            nonrobust   LLR p-value:                 3.689e-49
                                                 coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------
const                                       

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X['pp'], y['pp'], test_size=0.1)
rf = RandomForestClassifier(random_state=0)
param_grid = { 
    'n_estimators': [50, 100, 250, 500],
    #'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [2,5,10,None]}
    #'oob_score': [True,False]}

CV_rfc = GridSearchCV(estimator=rf, n_jobs=-1, param_grid=param_grid, verbose=10, scoring='neg_log_loss',cv= 5)
CV_rfc.fit(X_train, y_train)
print(CV_rfc.best_params_, CV_rfc.best_score_)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   22.5s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   24.6s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   30.5s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   39.0s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   48.3s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:  1.5min finished


{'max_depth': 5, 'n_estimators': 500} -0.560280774844516


## Overall

Flights that appear in each of the three apps

In [125]:
#redundant columns check
matrix = (X['all']).corr()==1
linear_dependents = []
for col in matrix.columns:
    #print(col)
    linear_dependents.append(list(matrix[col].index[matrix[col]==True]))
[x for x in linear_dependents if len(x)>1]

[['wac-oauthmsatokenretrieval-', 'wac-oauthtestappmsatokenretrieval-'],
 ['wac-oauthmsatokenretrieval-', 'wac-oauthtestappmsatokenretrieval-']]

In [126]:
#opposite columns check
matrix = (X['all']).corr()==-1
linear_combinations = []
for col in matrix.columns:
    #print(col)
    linear_combinations.append(list(matrix[col].index[matrix[col]==True]))
[x for x in linear_combinations if len(x)>1]

[]

In [127]:
rank_test = (sm.add_constant(X['all'].T.drop_duplicates().T))
for i in range(rank_test.shape[1]):
    df_to_rank = rank_test.iloc[:,:i+1]
    print(i,np.linalg.matrix_rank(df_to_rank))

0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10
10 11
11 12
12 13
13 14
14 15
15 16
16 17
17 18
18 19
19 20
20 21
21 22
22 23
23 24
24 25
25 26
26 27
27 28
28 29
29 30
30 31
31 32
32 33
33 34
34 35
35 36
36 37
37 38
38 39
39 40
40 41


In [128]:
logit = sm.Logit(y['all'],sm.add_constant(X['all'].T.drop_duplicates().T))
flogit = logit.fit()
print(flogit.summary())
coefficients = flogit.summary2().tables[1]
coefficients = coefficients[coefficients['P>|z|']<0.1]
coefficients['Odds Ratio']=np.exp(coefficients['Coef.'])
coefficients['O.R.LB']=np.exp(coefficients['[0.025'])
coefficients['O.R.UB']=np.exp(coefficients['0.975]'])
coefficients['FinalProbability'] = np.exp(coefficients['Coef.'].round(1))*y['all'].value_counts(normalize=True).loc[1] #- 0.5
coefficients['FinalProbability'] = coefficients['FinalProbability'].mask(coefficients['FinalProbability']>=1,0.99)
coefficients['Lift'] = coefficients['FinalProbability'] - y['all'].value_counts(normalize=True).loc[1]
all_coefficients = coefficients.join(flight_durations)
all_coefficients['Type'] = 'All'

Optimization terminated successfully.
         Current function value: 0.600190
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                    NPS   No. Observations:               112240
Model:                          Logit   Df Residuals:                   112199
Method:                           MLE   Df Model:                           40
Date:                Wed, 15 Jan 2020   Pseudo R-squ.:                 0.01482
Time:                        17:24:52   Log-Likelihood:                -67365.
converged:                       True   LL-Null:                       -68379.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                                                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
const     

## Scorecards

In [199]:
excel_coefficients = excel_coefficients.round(2)
word_coefficients = word_coefficients.round(2)
pp_coefficients = pp_coefficients.round(2)
all_coefficients = all_coefficients.round(2)

In [200]:
excel_coefficients['TreatmentCount'] = X['excel'].sum()[excel_coefficients.index]
excel_coefficients

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975],Odds Ratio,O.R.LB,O.R.UB,FinalProbability,Lift,FlightStart,FlightEnd,FlightCounts,Type,TreatmentCount
const,1.03,0.04,26.62,0.00,0.96,1.11,2.81,2.61,3.04,0.99,0.30,NaN,NaN,NaN,Excel,NaN
xls-editablelegacycomments-,0.28,0.04,6.84,0.00,0.20,0.36,1.32,1.22,1.43,0.93,0.24,2019-07-16 19:19:12,2019-12-30 23:44:09,8322.0,Excel,7938.0
xls-earlysessionstartforview-,-0.09,0.04,-2.43,0.02,-0.17,-0.02,0.91,0.84,0.98,0.62,-0.07,2019-07-07 12:25:22,2019-12-30 23:59:06,10227.0,Excel,5993.0
xls-ribbonresourcespreloading-,0.10,0.04,2.65,0.01,0.03,0.17,1.10,1.03,1.18,0.76,0.07,2019-05-02 17:25:05,2019-12-30 23:44:09,14745.0,Excel,7256.0
xls-tabletextcontrastacccheck-,-0.46,0.04,-10.78,0.00,-0.55,-0.38,0.63,0.58,0.68,0.42,-0.27,2019-05-01 02:10:10,2019-07-15 18:10:07,5713.0,Excel,5644.0
xls-xlowatermarkroundtrip-,-0.14,0.04,-3.82,0.00,-0.21,-0.07,0.87,0.81,0.93,0.62,-0.07,2019-09-18 06:00:48,2019-12-28 17:09:06,17185.0,Excel,15354.0
xls-ideas-,-0.17,0.03,-5.20,0.00,-0.23,-0.11,0.84,0.79,0.90,0.56,-0.12,2019-05-01 09:20:20,2019-12-30 23:59:06,29988.0,Excel,29271.0
xls-newcopypasteexperienceacrosssessions-,0.15,0.04,3.55,0.00,0.07,0.23,1.16,1.07,1.26,0.76,0.07,2019-05-01 02:59:06,2019-11-11 13:09:14,7550.0,Excel,7042.0
xls-reconnectsessiononuserinteration-,-0.71,0.21,-3.38,0.00,-1.12,-0.30,0.49,0.33,0.74,0.34,-0.35,2019-08-04 12:29:06,2019-08-15 10:15:38,209.0,Excel,100.0
xls-applyautomaticpolicylabel-,-0.11,0.04,-3.04,0.00,-0.18,-0.04,0.90,0.84,0.96,0.62,-0.07,2019-10-30 23:00:16,2019-12-30 23:59:06,9550.0,Excel,8970.0


In [201]:
word_coefficients['TreatmentCount'] = X['word'].sum()[word_coefficients.index]
word_coefficients

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975],Odds Ratio,O.R.LB,O.R.UB,FinalProbability,Lift,FlightStart,FlightEnd,FlightCounts,Type,SelectedBy,TreatmentCount
const,0.96,0.02,42.18,0.00,0.91,1.00,2.61,2.49,2.73,0.99,0.30,NaN,NaN,NaN,Word,[],NaN
wac-wordnotificationbarimageerror-,0.64,0.33,1.95,0.05,-0.00,1.27,1.89,1.00,3.57,0.99,0.30,2019-12-19 19:09:08,2019-12-30 21:25:12,125.0,Word,[Recursive],63.0
wac-wordsupportimagedragdropexternal-,-0.53,0.28,-1.88,0.06,-1.07,0.02,0.59,0.34,1.02,0.42,-0.27,2019-12-19 18:04:08,2019-12-30 21:10:11,133.0,Word,[Recursive],56.0
wac-wordideas-,0.33,0.12,2.87,0.00,0.11,0.56,1.39,1.11,1.75,0.93,0.24,2019-07-14 01:45:06,2019-12-30 23:40:16,12081.0,Word,[RandomForest],11307.0
wac-autoclpisenabled-,-0.15,0.04,-4.16,0.00,-0.22,-0.08,0.86,0.80,0.92,0.62,-0.07,2019-10-24 16:05:10,2019-12-30 23:40:16,18857.0,Word,[CHI2],11760.0
wac-wordreacttaskpaneinfrastructure-,0.27,0.06,4.58,0.00,0.15,0.38,1.31,1.17,1.46,0.93,0.24,2019-05-16 00:39:21,2019-10-15 22:05:04,7994.0,Word,[CHI2],7642.0
wac-wordreacttaskpanewithmlr-,0.17,0.09,1.79,0.07,-0.02,0.35,1.18,0.98,1.42,0.84,0.15,2019-05-16 13:34:08,2019-09-20 17:39:49,5439.0,Word,[CHI2],5258.0
wac-box4augloopseedingzerosequence-,0.14,0.05,2.70,0.01,0.04,0.23,1.14,1.04,1.26,0.76,0.07,2019-09-11 19:40:13,2019-12-30 23:40:16,26443.0,Word,[F],25342.0
wac-wordkeepmaxlastknownheight-,-0.17,0.07,-2.22,0.03,-0.31,-0.02,0.85,0.73,0.98,0.56,-0.12,2019-09-19 18:34:06,2019-12-30 23:59:06,69413.0,Word,"[RandomForest, F]",23775.0


In [202]:
pp_coefficients['TreatmentCount'] = X['pp'].sum()[pp_coefficients.index]
pp_coefficients

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975],Odds Ratio,O.R.LB,O.R.UB,FinalProbability,Lift,FlightStart,FlightEnd,FlightCounts,Type,TreatmentCount
const,1.07,0.04,25.92,0.00,0.98,1.15,2.90,2.68,3.14,0.99,0.25,NaN,NaN,NaN,PowerPoint,NaN
ppt-appcommandsquickcheck-,0.16,0.06,2.64,0.01,0.04,0.28,1.17,1.04,1.32,0.90,0.16,2019-10-16 02:41:42,2019-12-30 22:35:07,4508.0,PowerPoint,3571.0
ppt-designerdashboardtest-,-0.10,0.06,-1.79,0.07,-0.21,0.01,0.91,0.81,1.01,0.67,-0.07,2019-05-01 00:04:11,2019-08-08 21:10:18,5941.0,PowerPoint,2993.0
ppt-formatpainterapplyremapped-,0.10,0.05,1.99,0.05,0.00,0.20,1.10,1.00,1.22,0.82,0.08,2019-07-04 02:04:08,2019-09-11 02:49:10,4782.0,PowerPoint,3948.0
ppt-freemiumupsellheaderui-,0.13,0.07,1.91,0.06,-0.00,0.27,1.14,1.00,1.31,0.82,0.08,2019-05-01 01:34:07,2019-12-30 23:44:09,4716.0,PowerPoint,1782.0
ppt-helptabwhatsnewenabled-,0.18,0.06,3.07,0.00,0.07,0.30,1.20,1.07,1.35,0.90,0.16,2019-05-31 04:59:17,2019-11-26 14:50:37,6796.0,PowerPoint,6577.0
ppt-pasteslidekeepsourceformattingdefault-,-0.12,0.05,-2.16,0.03,-0.22,-0.01,0.89,0.80,0.99,0.67,-0.07,2019-09-17 17:55:11,2019-12-30 23:15:10,15642.0,PowerPoint,13261.0
ppt-pptsharedcommentsfluentui-,-0.14,0.04,-3.50,0.00,-0.22,-0.06,0.87,0.81,0.94,0.67,-0.07,2019-08-21 14:34:14,2019-11-14 10:26:02,10243.0,PowerPoint,7356.0
ppt-sendshuttingdownbeaconrequests-,-0.13,0.07,-1.81,0.07,-0.28,0.01,0.87,0.75,1.01,0.67,-0.07,2019-09-24 03:00:09,2019-11-15 09:36:46,1955.0,PowerPoint,967.0
ppt-slideeditforaugmentationloop-,-0.16,0.05,-3.22,0.00,-0.26,-0.06,0.85,0.77,0.94,0.61,-0.13,2019-11-14 12:10:05,2019-12-30 23:15:10,7100.0,PowerPoint,6280.0


In [203]:
all_coefficients['TreatmentCount'] = X['all'].sum()[all_coefficients.index]
all_coefficients

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975],Odds Ratio,O.R.LB,O.R.UB,FinalProbability,Lift,FlightStart,FlightEnd,FlightCounts,Type,TreatmentCount
const,1.12,0.02,53.86,0.00,1.08,1.16,3.06,2.93,3.18,0.99,0.29,NaN,NaN,NaN,All,NaN
wac-wordimagegroupcontainerfittopage-,0.09,0.04,2.12,0.03,0.01,0.17,1.09,1.01,1.19,0.78,0.07,2019-10-09 00:05:40,2019-12-02 22:15:18,12006.0,All,10610.0
wac-wordpreserveverbonredirect-,-0.20,0.03,-7.99,0.00,-0.25,-0.15,0.82,0.78,0.86,0.57,-0.13,2019-05-01 00:04:11,2019-08-12 15:19:05,21598.0,All,17817.0
firstrelease-,0.17,0.02,7.52,0.00,0.12,0.21,1.18,1.13,1.23,0.86,0.16,2019-05-01 00:04:11,2019-12-30 23:44:09,35100.0,All,17199.0
wac-wordcloneatmentionpropertiesinhyperlink-,-0.04,0.02,-1.69,0.09,-0.09,0.01,0.96,0.92,1.01,0.70,0.00,2019-10-24 01:20:15,2019-12-30 23:34:09,19778.0,All,15427.0
wac-licensingisenabled-,-0.20,0.02,-10.40,0.00,-0.24,-0.16,0.82,0.79,0.85,0.57,-0.13,2019-07-15 17:24:20,2019-12-30 23:59:06,44376.0,All,42744.0
wac-wordsdxpreloadpackagelist-,0.08,0.02,3.95,0.00,0.04,0.13,1.09,1.04,1.13,0.78,0.07,2019-09-09 18:09:56,2019-11-11 13:09:14,23545.0,All,20466.0
wac-wordwordoauthtestappaadtokenretrieval-,-0.08,0.02,-3.86,0.00,-0.11,-0.04,0.93,0.89,0.96,0.64,-0.07,2019-06-12 12:04:04,2019-12-30 23:40:16,31840.0,All,15363.0
wac-wordshowpasteoptionsincontextmenu-,0.14,0.05,2.64,0.01,0.04,0.25,1.15,1.04,1.28,0.78,0.07,2019-07-08 18:14:05,2019-10-07 10:04:14,4384.0,All,2208.0
wac-oauthmsatokenretrieval-,-0.43,0.16,-2.80,0.01,-0.74,-0.13,0.65,0.48,0.88,0.47,-0.23,2019-10-14 19:21:14,2019-12-30 23:44:09,13549.0,All,6414.0


In [204]:
pp_coefficients.head(1)

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975],Odds Ratio,O.R.LB,O.R.UB,FinalProbability,Lift,FlightStart,FlightEnd,FlightCounts,Type,TreatmentCount
const,1.07,0.04,25.92,0.0,0.98,1.15,2.9,2.68,3.14,0.99,0.25,NaN,NaN,NaN,PowerPoint,NaN


In [205]:
scorecard = pd.concat([excel_coefficients, word_coefficients,pp_coefficients,all_coefficients])

C:\Users\v-Jaand\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [206]:
scorecard = scorecard.drop('const')

In [207]:
scorecard['TreatmentShare'] = (scorecard['TreatmentCount']/scorecard['FlightCounts']).round(2)

In [211]:
scorecard = scorecard.reindex(columns = ['Type','FlightStart','FlightEnd','FlightCounts','TreatmentCount',
                     'TreatmentShare','FinalProbability','Lift','Coef.','Std.Err.','z',
                     'P>|z|','[0.025','0.975]','Odds Ratio','O.R.LB','O.R.UB'])

In [212]:
scorecard.to_csv('ABModelFlightScorecard('+month+').csv')

In [6]:
month = 'December'
scorecard = pd.read_csv('ABModelFlightScorecard('+month+').csv',index_col=0)

In [8]:
scorecard.head()

,Type,FlightStart,FlightEnd,FlightCounts,TreatmentCount,TreatmentShare,FinalProbability,Lift,Coef.,Std.Err,z,P>|z|,[0.025,0.975],Odds Ratio,O.R.LB,O.R.UB
xls-editablelegacycomments-,Excel,7/16/2019 19:19,12/30/2019 23:44,8322,7938,0.95,0.93,0.24,0.28,NaN,6.84,0.00,0.20,0.36,1.32,1.22,1.43
xls-earlysessionstartforview-,Excel,7/7/2019 12:25,12/30/2019 23:59,10227,5993,0.59,0.62,-0.07,-0.09,NaN,-2.43,0.02,-0.17,-0.02,0.91,0.84,0.98
xls-ribbonresourcespreloading-,Excel,5/2/2019 17:25,12/30/2019 23:44,14745,7256,0.49,0.76,0.07,0.10,NaN,2.65,0.01,0.03,0.17,1.10,1.03,1.18
xls-tabletextcontrastacccheck-,Excel,5/1/2019 2:10,7/15/2019 18:10,5713,5644,0.99,0.42,-0.27,-0.46,NaN,-10.78,0.00,-0.55,-0.38,0.63,0.58,0.68
xls-xlowatermarkroundtrip-,Excel,9/18/2019 6:00,12/28/2019 17:09,17185,15354,0.89,0.62,-0.07,-0.14,NaN,-3.82,0.00,-0.21,-0.07,0.87,0.81,0.93


In [18]:
for m in ['December','November','October']:
    start = pd.read_csv('ABModelFlightScorecard('+month+').csv',
                        index_col=0).FlightStart.to_dict()
    scorecard['FlightStart'] = scorecard.index.map(start)


In [19]:
scorecard.to_csv()

xls-editablelegacycomments-                                       7/16/2019 19:19
xls-earlysessionstartforview-                                      7/7/2019 12:25
xls-ribbonresourcespreloading-                                     5/2/2019 17:25
xls-tabletextcontrastacccheck-                                      5/1/2019 2:10
xls-xlowatermarkroundtrip-                                         9/18/2019 6:00
xls-ideas-                                                          5/1/2019 9:20
xls-newcopypasteexperienceacrosssessions-                           5/1/2019 2:59
xls-reconnectsessiononuserinteration-                              8/4/2019 12:29
xls-applyautomaticpolicylabel-                                   10/30/2019 23:00
xls-ocpsisenabled-                                                10/7/2019 19:09
xls-namedsheetviewspassivetest-                                    5/2/2019 14:35
xls-linkedentityblankcellconvertteachingcalloutvideowithtext-     6/27/2019 14:55
xls-slrcachecomb

In [21]:
scorecard.to_csv('ABModelFlightScorecard('+month+')Updated.csv')